<a href="https://colab.research.google.com/github/nipdep/SpCup-22-Init/blob/team1/src/sbcnn/param_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Neptune Tracker](https://app.neptune.ai/chalani.ekanayake212/SBCNN-SPCUP/experiments?split=tbl&dash=charts&viewId=standard-view&sortBy=%5B%22sys%2Fcreation_time%22%5D&sortFieldType=%5B%22datetime%22%5D&sortFieldAggregationMode=%5B%22auto%22%5D&sortDirection=%5B%22descending%22%5D&suggestionsEnabled=false&lbViewUnpacked=true)

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
%cd /gdrive/MyDrive/SP Cup 2022/SpCup-22-Init/src/sbcnn

/gdrive/MyDrive/SP Cup 2022/SpCup-22-Init/src/sbcnn


In [3]:
! pip install neptune-client &> /dev/null

In [4]:
!pip install audiomentations &> /dev/null

In [5]:
!pip install pedalboard &> /dev/null

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import Callback
import warnings
import neptune.new as neptune
from dataloader_v1 import SpectDataset
import numpy as np
from sklearn.metrics import classification_report
import itertools
from getpass import getpass
import pandas as pd
import os

warnings.filterwarnings('ignore')

In [7]:
api_token = getpass('Enter your private Neptune API token: ')

Enter your private Neptune API token: ··········


In [8]:
workspace = 'chalani.ekanayake212' 
project = 'SBCNN-SPCUP'
project_name = workspace + '/' + project

In [9]:
run = neptune.init(project=project_name, name="RMS, 0.001 ILR", api_token=api_token)

https://app.neptune.ai/chalani.ekanayake212/SBCNN-SPCUP/e/SBCNNSPC-31
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [10]:
#! wget "https://www.dropbox.com/s/36yqmymkva2bwdi/spcup_2022_training_part1.zip?dl=1" -c -O '/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_part1.zip'
#! wget "https://www.dropbox.com/s/wsmlthhri29fb79/spcup_2022_unseen.zip?dl=1" -c -O '/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen.zip'

In [11]:
#!unzip "/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_part1.zip" -d "/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_data/" > /dev/null
#!unzip "/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen.zip" -d "/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen_data/" > /dev/null

In [12]:
# create combined label df
# combine two label sets
#import pandas as pd
#df1 = pd.read_csv('/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_data/spcup_2022_training_part1/labels.csv')
#df2 = pd.read_csv('/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen_data/spcup_2022_unseen/labels.csv')
#df3 = pd.concat([df1, df2]).sample(frac=1)

#df3.to_csv('/gdrive/MyDrive/SP Cup 2022/final_labels.csv', index=False)
#!rm '/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen_data/spcup_2022_unseen/labels.csv'

In [13]:
# copy .wav files from unseen to all
#!cp -a "/gdrive/MyDrive/SP Cup 2022/spcup_2022_unseen_data/spcup_2022_unseen/". "/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_data/spcup_2022_training_part1/"

In [14]:
def SBCNN_Model(input_shape, num_labels):
    model = Sequential()
    model.add(Input(input_shape))
    model.add(Conv2D(24, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='valid'))
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_regularizer=L2(0.001)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_labels, kernel_regularizer=L2(0.001)))
    model.add(Activation('softmax'))

    return model

In [15]:
DATA_PATH = '/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_data/spcup_2022_training_part1/'
Labeled_dir = '/gdrive/MyDrive/SP Cup 2022/final_labels.csv'

In [16]:
labels_df = pd.read_csv(Labeled_dir)
labels_df.head()

,track,algorithm
0,060d7d1bbcaf59f7f0e37a18117ddbda.wav,2
1,2f9cfbe869595c7deefc3fb184cb71cb.wav,5
2,d1b370d1dc59b54f0d7961e3d995e97f.wav,3
3,817fe3a19b02f115c9e77a4ca4639925.wav,3
4,538f6d6b14d3eef3452872b955a43391.wav,5


In [17]:
labels_df['algorithm'].value_counts()

2    1000
5    1000
3    1000
0    1000
4    1000
1    1000
Name: algorithm, dtype: int64

In [18]:
print(f"There are {len(os.listdir(DATA_PATH))} files of audio for training.\n")

There are 6000 files of audio for training.



In [19]:
def run_model(optimizer, n_iter):
        
    DATA_PATH = '/gdrive/MyDrive/SP Cup 2022/spcup_2022_training_data/spcup_2022_training_part1/'
    
    Labeled_dir = '/gdrive/MyDrive/SP Cup 2022/final_labels.csv'

    dataloader = SpectDataset(add_noise=True, reverb=True)
    train_ds, val_ds = dataloader.call(DATA_PATH, Labeled_dir)

    for spectrogram, _ in train_ds.take(1):
        input_shape = spectrogram.shape

    model = SBCNN_Model(input_shape[1:], 6)

    class PrintLRCallback(Callback):
      def on_epoch_end(self, epoch, logs={}):
        print("\nSBCNN with learning rate {} \n".format(model.optimizer._decayed_lr('float32').numpy()))
        for log_name, log_value in logs.items():
            run["epoch/{}".format(log_name)].log(log_value)
            run["epoch/{}".format('Learning Rate')].log(model.optimizer._decayed_lr('float32').numpy())

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )

    EPOCHS = n_iter
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        verbose=2,
        callbacks=[PrintLRCallback()]
    # callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )

    model.save("/gdrive/MyDrive/SP Cup 2022/SBCNN Models/SBCNN_RMS_31.h5")
    print("Saved model to disk")

    res_sparse = model.predict(val_ds)
    res_np = np.argmax(res_sparse, axis=1)
    pred_labels = list(res_np)
    true_labels = []
    for _, label in val_ds.take(1000):
        true_labels.extend(list(label.numpy()))

   # class_names = [f'class_{i}' for i in range(6)]
   # print(classification_report(true_labels, pred_labels, target_names=class_names))

In [20]:
initial_lr = 0.1
decay_r = 0.9

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
              initial_learning_rate=initial_lr,
              decay_steps=(1000), 
              decay_rate=decay_r)

sgd = SGD(learning_rate = lr_schedule)
adm = Adam(learning_rate=initial_lr)
rms = tf.keras.optimizers.RMSprop(learning_rate=0.001)

n_iter = 20

optimizer = rms

# log params
run["parameters/epochs"] = n_iter
run["parameters/optimizer"] = "RMS"
run["parameters/initial_lr"] = 0.001
run["parameters/decay_rate"] = 'none'

run_model(optimizer, n_iter)

Epoch 1/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 69s - loss: 1.2651 - accuracy: 0.5158 - val_loss: 0.7132 - val_accuracy: 0.7600 - 69s/epoch - 461ms/step
Epoch 2/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 2s - loss: 0.8056 - accuracy: 0.7092 - val_loss: 0.5946 - val_accuracy: 0.8075 - 2s/epoch - 13ms/step
Epoch 3/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 2s - loss: 0.6582 - accuracy: 0.7765 - val_loss: 0.5121 - val_accuracy: 0.8333 - 2s/epoch - 13ms/step
Epoch 4/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 2s - loss: 0.5625 - accuracy: 0.8250 - val_loss: 0.4514 - val_accuracy: 0.8592 - 2s/epoch - 17ms/step
Epoch 5/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 2s - loss: 0.4943 - accuracy: 0.8531 - val_loss: 0.4285 - val_accuracy: 0.8767 - 2s/epoch - 13ms/step
Epoch 6/20

SBCNN with learning rate 0.0010000000474974513 

150/150 - 2s - loss: 0.4402 - accuracy: 0.8740 - val_loss: 0.4198 - val

In [21]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 14 operations to synchronize with Neptune. Do not kill this process.


All 14 operations synced, thanks for waiting!
